In [1]:
# Configuration for auto-completion
%config Completer.use_jedi = False
%config Completer.greedy = True

# Enable test mode for CI pipeline
import sys
if len(sys.argv) == 2 and sys.argv[1] == "test":
    test_mode = True
    server = "https://qc-plat-sta.dlr.de/"
else:
    test_mode = False
    server = "https://connect.qci.dlr.de/"


In [2]:
import time
from qciconnect import Client

# Load environment variables for logging in to QCI Connect
# Insert your personal access token below. (You need to create it on the QCI Connect website on your account page.)
token=%env CI_QCICONNECT_TOKEN

In [3]:
client = Client(token=token, server=server)

In [4]:
client.qpus.show()

Alias                    Qubits  Status       Technology             Accessible to me
---------------------  --------  -----------  ---------------------  ------------------
qsea_i_digital_twin          10  idle         digital_twin           True
dedalo                       64  coming_soon  photons                True
qsea_ii                      20  coming_soon  trapped_ions           True
comiqc                       50  coming_soon  solid_state_spins      True
xq2i                          8  coming_soon  vacancy_centres        True
redac                         8  coming_soon  analogue_computer      True
dinaqc                      100  coming_soon  neutral_atoms          True
legato                       20  coming_soon  trapped_ions           True
toccata                      50  coming_soon  trapped_ions           True
xaphiro                      50  coming_soon  trapped_ions           True
sunqc                         8  coming_soon  vacancy_centres        True
qsea_i      

In [ ]:
client.qpus.xq1i.show()

## Compile Job Submission

In [ ]:
circuit = """OPENQASM 3.0;
include "stdgates.inc";
qubit[2] q;
h q[0];
cx q[0], q[1];"""

In [ ]:
result = client.methods.transpilation_qiskit_compiler({"circuit": circuit})

In [ ]:
print(result.compiled_circuit)

In [ ]:
result = client.methods.transpilation_qiskit_compiler({"circuit": circuit}, {
      "qubit_connectivity": [[0,1]],
      "gate_set": ["h", "x", "cz"],
      "optimization_level": 2
    })

In [ ]:
print(result.compiled_circuit)

## Non-blocking compile job submission

In [ ]:
promise = client.methods.transpilation_qiskit_compiler({"circuit": circuit}, wait_for_results=False)


In [ ]:
if test_mode:
    print("Waiting in test mode for job to complete...")
    time.sleep(5)
    
result = promise.result

In [ ]:
print(result.compiled_circuit)

## Retrieve result by job ID

In [ ]:
result = client.results[promise.job_id]

In [ ]:
print(result.compiled_circuit)

## QPU Job Submission

In [ ]:
client.qpus.show()

In [ ]:
result = client.qpus.statevector_simulator.submit(circuit, primitive="sampling", shots=10, name="API Demonstration", comment="Submit algorithm X with parameters YZ")

In [ ]:
print(result.data)

## Non-blocking QPU job submission

In [ ]:
promise = client.qpus.statevector_simulator.submit(circuit, primitive="quantum_state", shots=10, wait_for_results=False, name="API Demonstration", comment="Submit algorithm X with parameters YZ")


In [ ]:
if test_mode:
    print("Waiting in test mode for job to complete...")
    time.sleep(5)
    
result = promise.result
print(result.data)

In [ ]:
if not test_mode:
    promise = client.qpus.saxonq_4qb_rt.submit(circuit, "sampling", name="Default circuit on SaxonQ machine", shots=600000, wait_for_results=False)
    

In [ ]:
if not test_mode:
    result = promise.result

In [ ]:
print(result.data)